<a href="https://colab.research.google.com/github/arjunmalik11/AI-Agents-with-Autogen/blob/main/Autogen_Customer_Onboarding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
OPENAI_API_KEY = "secret-key"
llm_config = {
    "model" : "gpt-4",
    "api_key" : OPENAI_API_KEY,
 }

In [ ]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [18]:
from autogen import ConversableAgent

onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding_Topic_preference_Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 09-11 15:31:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-11 15:31:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 09-11 15:31:42] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [19]:
from autogen import initiate_chats

chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": (
            "Hello, I'm here to help you get started with our product. "
            "Could you tell me your name and location?"
        ),
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt": "Return the customer information as a JSON object only: "
                              "{\"name\": \"\", \"location\": \"\"}",
        },
        "max_turns": 2,
        "clear_history": True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": (
            "Great! Could you tell me what topics you are "
            "interested in reading about?"
        ),
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history": False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm"
    }
]

chat_results = initiate_chats(chats)



********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product. Could you tell me your name and location?

--------------------------------------------------------------------------------
Replying as customer_proxy_agent. Provide feedback to Onboarding_Personal_Information_Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Frank and Austin
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

Frank and Austin

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Nice to meet you, Frank from Austin. Welcome to our community! TERMINATE

---------------------------------------------------------------------

In [21]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{"name": "Frank", "location": "Austin"}


The user, Frank, is interested in reading about the topic of music.


The user, Frank, is interested in music and might appreciate reading fun facts or humor related to this topic. Since he is located in Austin, Texas, known as the Live Music Capital of the World, music-related content connected to his location may also be of interest.




In [22]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00849, 'gpt-4-0613': {'cost': 0.00849, 'prompt_tokens': 227, 'completion_tokens': 28, 'total_tokens': 255}}, 'usage_excluding_cached_inference': {'total_cost': 0.00849, 'gpt-4-0613': {'cost': 0.00849, 'prompt_tokens': 227, 'completion_tokens': 28, 'total_tokens': 255}}}


{'usage_including_cached_inference': {'total_cost': 0.0031199999999999995, 'gpt-4-0613': {'cost': 0.0031199999999999995, 'prompt_tokens': 74, 'completion_tokens': 15, 'total_tokens': 89}}, 'usage_excluding_cached_inference': {'total_cost': 0.0031199999999999995, 'gpt-4-0613': {'cost': 0.0031199999999999995, 'prompt_tokens': 74, 'completion_tokens': 15, 'total_tokens': 89}}}


{'usage_including_cached_inference': {'total_cost': 0.02841, 'gpt-4-0613': {'cost': 0.02841, 'prompt_tokens': 411, 'completion_tokens': 268, 'total_tokens': 679}}, 'usage_excluding_cached_inference': {'total_cost': 0.02841, 'gpt-4-0613': {'cost': 0.02841, 'prompt_tokens': 411, 'completion_toke